<a href="https://colab.research.google.com/github/deepcodesss/Sentiment-Analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# installing kaggle library
! pip install kaggle

In [7]:
 # configuring the path of Kaggle.json file
 !mkdir -p ev ~/.kaggle
 !cp kaggle.json ~/.kaggle/
 !chmod 600 ~/.kaggle/kaggle.json

In [8]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.26GB/s]


In [9]:
# Extracting the compressed zip file

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


In [10]:
# Importing the dependencies

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
# Printing the stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [13]:
# Data Processing
# Loading the data from csv file from pandas csv file
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [14]:
# Checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [15]:
# Printing the first five rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [16]:
# Namibg the columns and reading the dataaset again
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1', names = column_names)

In [17]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
twitter_data.shape

(1600000, 6)

In [19]:
# Counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [20]:
# Checking the distribution of the target values
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [21]:
# Convert the target from 4 to 1
twitter_data.replace({'target' : {4:1}}, inplace = True)

In [22]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [23]:
# 0 ----> Negative Tweet
# 1 ----> Positive Tweet

In [24]:
# Stemming - to convert a word to its root word
# example ---> actor, acting , actress = act

In [25]:
port_stem = PorterStemmer()

In [26]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [27]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [28]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [29]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [30]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [31]:
# Separating the data and label

X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [32]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [33]:
print(Y)

[0 0 0 ... 1 1 1]


In [34]:
# Splitting the data into training and testing data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2)

In [35]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [36]:
print(Y.shape, Y_train.shape, Y_test.shape)

(1600000,) (1280000,) (320000,)


In [37]:
# Converting the textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [38]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9452454 stored elements and shape (1280000, 461018)>
  Coords	Values
  (0, 31233)	0.7519171067329923
  (0, 129956)	0.6592576617698773
  (1, 241525)	0.5942540519072339
  (1, 437910)	0.8042773910733999
  (2, 237072)	0.6367700241935805
  (2, 42838)	0.2294043092907454
  (2, 128288)	0.2817039475788181
  (2, 93639)	0.16612263205577085
  (2, 416219)	0.2970321685021027
  (2, 140234)	0.2237237868613854
  (2, 145727)	0.16444046079173227
  (2, 278606)	0.48531397707654644
  (2, 411086)	0.18464287904169108
  (3, 201928)	0.524207488360723
  (3, 308910)	0.41292329558269175
  (3, 179097)	0.4788299006944713
  (3, 445747)	0.2188627839232472
  (3, 8560)	0.37668420201866026
  (3, 286683)	0.3682809811482607
  (4, 457632)	0.7604081439946883
  (4, 302738)	0.35966531233864635
  (4, 315347)	0.3291134109458034
  (4, 415818)	0.27525076285252903
  (4, 227660)	0.3291530007938817
  (5, 93639)	0.181548748438646
  :	:
  (1279996, 132233)	0.2452899533029938

In [39]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289193 stored elements and shape (320000, 461018)>
  Coords	Values
  (0, 5622)	0.3011990126277844
  (0, 53069)	0.3855937954496085
  (0, 149573)	0.10062096511841492
  (0, 177712)	0.12894291546244904
  (0, 220160)	0.1262929409556629
  (0, 272353)	0.31389655689807383
  (0, 280262)	0.4118569164941687
  (0, 298811)	0.13515991209574565
  (0, 313583)	0.336215114519969
  (0, 317487)	0.18257148054132444
  (0, 334325)	0.1834654984968235
  (0, 371823)	0.2094373271360439
  (0, 387006)	0.1617603264881936
  (0, 414670)	0.18963151083555396
  (0, 428015)	0.3406334479403277
  (0, 445807)	0.17151168445800502
  (1, 30475)	0.6048761579069399
  (1, 154446)	0.7963195549499829
  (2, 39327)	0.3345556683317226
  (2, 43578)	0.434390421943326
  (2, 306745)	0.4614836588658659
  (2, 344198)	0.45616935735592273
  (2, 384696)	0.5275602488060214
  (3, 57543)	0.7486009460984386
  (3, 218938)	0.5023157649783742
  :	:
  (319995, 451190)	0.5325229312386054
  

In [40]:
# Training the Machine Learning Model

# Logistic Regression

model = LogisticRegression(max_iter = 1000)

In [41]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [42]:
# Model Evaluation

# Accuracy Score

# Accuracy score of the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [43]:
print('Accuracy score on the training data', training_data_accuracy)

Accuracy score on the training data 0.80238671875


In [44]:
# Accuracy score of the testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [45]:
print('Accuracy score on the testing data', testing_data_accuracy)

Accuracy score on the testing data 0.77656875


In [46]:
# Saving the model
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [47]:
# Using the saved model for prediction

loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [48]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[0]
Negative Tweet


In [49]:
X_new = X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[1]
Positive Tweet


In [50]:
# ------- Project Complete -------- #